# Importing external sequences

In [ ]:
from opencloning.ncbi_requests import get_annotations_from_query, get_genome_region_from_annotation, get_genbank_sequence
from pydna.opencloning_models import CloningStrategy

## Importing genomic sequences

You can import genomic sequences in different ways

### Querying the annotation of a genome

Let's start by querying all the annotations of the _S. cerevisiae_ genome that contain the word "aldolase".

We are using the assembly accession of the reference genome, which is `GCF_000146045.2`. If you want to find sequence accessions for your genome of interest, you can use the [NCBI Datasets page](https://www.ncbi.nlm.nih.gov/datasets/genome/?taxon=460519).

> Remember that this is an asynchronous function, so you need to use `await` to call it inside notebooks or async functions, and `asyncio.run` to call it in normal scripts.

In [ ]:
annotations = await get_annotations_from_query('aldolase', 'GCF_000146045.2')

Now let's check what the annotation contains. It has a lot of info!

In [ ]:
annotations[0]

{'gene_id': '851888',
 'symbol': 'DPL1',
 'name': 'sphinganine-1-phosphate aldolase DPL1',
 'gene_type': 'protein-coding',
 'locus_tag': 'YDR294C',
 'genomic_regions': [{'gene_range': {'accession_version': 'NC_001136.10',
    'range': [{'begin': '1050459',
      'end': '1052228',
      'orientation': 'minus'}]}}],
 'transcripts': [{'accession_version': 'NM_001180602.1',
   'name': 'sphinganine-1-phosphate aldolase DPL1',
   'cds': {'accession_version': 'NM_001180602.1'},
   'genomic_locations': [{'genomic_accession_version': 'NC_001136.10',
     'genomic_range': {'begin': '1050459',
      'end': '1052228',
      'orientation': 'minus'}}],
   'protein': {'accession_version': 'NP_010580.1',
    'name': 'sphinganine-1-phosphate aldolase DPL1',
    'length': 589}}],
 'chromosomes': ['IV'],
 'annotations': [{'assembly_accession': 'GCF_000146045.2'}]}

Let's see what annotations we got:

In [ ]:
for i, annotation in enumerate(annotations):
    print(f'{i}: {annotation["locus_tag"]} - {annotation["name"]}')


0: YDR294C - sphinganine-1-phosphate aldolase DPL1
1: YEL046C - threonine aldolase GLY1
2: YER010C - bifunctional 4-hydroxy-4-methyl-2-oxoglutarate aldolase/oxaloacetate decarboxylase
3: YGR043C - sedoheptulose-7-phosphate:D-glyceraldehyde-3-phosphate transaldolase NQM1
4: YKL060C - fructose-bisphosphate aldolase FBA1
5: YLR354C - sedoheptulose-7-phosphate:D-glyceraldehyde-3-phosphate transaldolase TAL1
6: YNL256W - trifunctional dihydropteroate synthetase/dihydrohydroxymethylpterin pyrophosphokinase/dihydroneopterin aldolase FOL1


Great! Now let's say that what we are interested in is the `threonine aldolase GLY1` gene.
We can get the sequence of this locus by using the `get_genome_region_from_annotation` function, we can provide padding to the left and right of the gene to also get neighbouring regions and not just the gene itself.

In [ ]:
gly1 = await get_genome_region_from_annotation(annotations[0], 1000, 1000)

We can also see what information is stored in the source of the sequence, and display it's history.

In [ ]:
print(print(gly1.source))
print(gly1.history())


input=[] repository_id='NC_001136.10' coordinates=SimpleLocation(ExactPosition(1049458), ExactPosition(1053228), strand=-1) assembly_accession='GCF_000146045.2' locus_tag='YDR294C' gene_id=851888
None
╙── YDR294C (Dseqrecord(-3770))
    └─╼ GenomeCoordinatesSource


You can also save it to a json file and open it in [OpenCloning](https://app.opencloning.org), and it would look like this:

<img src="images/gly1_opencloning.png" width="300"/>

In [ ]:
cs = CloningStrategy.from_dseqrecords([gly1])

with open("gly_history.json", "w") as f:
    f.write(cs.model_dump_json())


### Using genome coordinates

For this, we need the sequence accession (not the assembly accession) of the chromosome of interest.

You can search for the sequence accession of interest in the [NCBI Datasets page](https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_000146045.2), at the `Chromosomes` section. You can also do this programmatically using the Datasets API, check how it is used in the function `get_sequence_accessions_from_assembly_accession`.

For our example, we are interested in chromosome IV, and the sequence accession is `NC_001136.10`. Let's take the same coordinates as before, and get the same gly1 sequence.


In [ ]:
gly1_from_coordinates = await get_genbank_sequence('NC_001136.10', start=1049459, end=1053228, strand=-1)

print("They are the same sequence:", gly1_from_coordinates.seq == gly1.seq)

They are the same sequence: True


Notice that in this case, the source will not contain gene id or locus tag.

In [ ]:
gly1_from_coordinates.source

NCBISequenceSource(input=[], repository_id='NC_001136.10', coordinates=SimpleLocation(ExactPosition(1049458), ExactPosition(1053228), strand=-1))